In [1]:
# pythonic version of the bash script instructions:
# for link to bash instructions, go to:
# https://gate.nmr.mgh.harvard.edu/wiki/bradmci/index.php/Individual_Reconstruction_%26_Analysis

# TO DO::
# - make sure the include does not run things again that have already been run
# - can I use regular expressions to just call _MT_ and _FPW_ ?
# FIX THE SAVING TO RAW FOLDER WHEN i figure out directory structure preferences (chnage RAW to simplified INCLUDE after regular expression implementation)
# enable: "issue a new project (each project has tasks connected to them__ BUT THESE TASKS COULD HAVE DIFFERENT PARAMS ETC>)"

In [2]:
def parse_findsession(findsession_output):
    session_values = {}
    for elements in findsession_output:
        try:
            [a, b] = elements.split(": ")
            a = a.strip()
            b = b.strip()
            if a in session_values:
                session_values[a].append(b)
            else:
                session_values[a]=[b]
        except ValueError:
            continue
    return session_values

In [3]:
def copyanything(src, dst):
    try:
        shutil.copytree(src, dst)
    except OSError as exc:
        if exc.errno == errno.ENOTDIR:
            shutil.copy(src, dst)
        else: raise

In [4]:
def maketxt(name, location):
    f = open(location+"subjectname", "w+")
    f.write(name)
    f.close()

In [5]:
def verifyrun(question):
    sys.stdout.write('%s [y/n]: ' %question)
    while True:
        try:
            return strtobool(input().lower())
        except ValueError:
            sys.stdout.write('Please respond with \'y\' or \'n\'.\n')
    

In [6]:
# load in all modules and pathways
from distutils.util import strtobool
import sys
import os
from os import system
from os.path import abspath

from nipype import Workflow, Node, MapNode, Function
from nipype.interfaces.fsl import BET, IsotropicSmooth, ApplyMask
from nipype.interfaces.freesurfer import MRIConvert

from nilearn.plotting import plot_anat
#%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import csv
import re
import shutil, errno

home = '/autofs/homes/002/rje11/'
mydirectory = '/autofs/cluster/animal/users/rezzo/'

# name after task--but must be contained in string in scaninfo file. This will be the title of the bold directory
task = ['FPW']
subname = '' #name of recon ## where to put this? how can I determine these values

unpackedtype = 'nii'

In [8]:
subjectlist = ['PCA_ITAY_F_34_20170110']

for indiv in subjectlist:

    findsession_output = !findsession PCA_ITAY_F_34_20170110
    cmdstring = 'findsession %s' % (indiv)
    system(cmdstring)
    print(cmdstring)

    # create dictionary with listed values (corresponding to each session)
    #session_values = parse_findsession(findsession_output)
    session_values = parse_findsession(findsession_output)
    
    find = re.compile(r"^[^_]*")
    projectname = re.search(find, indiv).group(0)
    
    try:
        # create a directory for unique session
        projectdir = mydirectory+projectname
        #projectdir = mydirectory+'projectname' # override with this until I know nomenclature
        os.makedirs(projectdir)
    except FileExistsError:
        print('PROJECT directory exists')
        continue

findsession PCA_ITAY_F_34_20170110
PROJECT directory exists


In [9]:
projectname

'PCA'

In [13]:
findsession_output

['=======',
 'SUBJECT:  PCA_ITAY_F_34_20171220',
 'SUBJ ID:  17.12.20-13:25:34-STD-1.3.12.2.1107.5.2.32.35162',
 'DATE   :  December 20, 2017',
 'TIME   :  13:41:04',
 'OPERATR:  mbrickho:pca (bay3) [D]',
 'XPRMNTR:  ',
 'PATH   :  /cluster/archive/319/siemens/TrioTim-35162-20171220-132641-001475']

In [10]:
session_values

{'SUBJECT': ['PCA_ITAY_F_34_20170110'],
 'SUBJ ID': ['17.01.10-12:58:44-STD-1.3.12.2.1107.5.2.32.35162'],
 'DATE': ['January 10, 2017'],
 'TIME': ['13:16:25'],
 'OPERATR': ['jcollins:ftl (bay3) [D]'],
 'XPRMNTR': [''],
 'PATH': ['/cluster/archive/317/siemens/TrioTim-35162-20170110-125934-001472']}

In [15]:
# for each session (5-10 min PER RUN)
for elements in range(0, len(session_values["SUBJECT"])): # define range as number of sessions
    srcdir = session_values["PATH"][elements]+"/"
    SESSION_DIR = projectdir+"/"+session_values["SUBJECT"][elements]+'/'
    dicomdir = SESSION_DIR+"dicoms/"
    taskname = session_values["SUBJECT"][elements]
    anatdir = SESSION_DIR+"anat/"
    
    try:
        # make directories for each session if not already existing
        os.makedirs(anatdir)
        os.makedirs(dicomdir)

        #load in the anatomical data\figure out best way to do this !!!!!!!!!!!!!
        anat_src = '/autofs/cluster/animal/users/rezzo/recons/'+ session_values["SUBJECT"][elements][4:] # temporary
        #recon = '/autofs/cluster/animal/scan_data/dl-xnat/MASTER/ITAY_F_34/PCA_ITAY_F_34_20171220/recons/FS6_004_006/2017-12-21T17-10-11-115/edit.FS6_004_006'
        cmdstring = "cp -a %s/* %s" % (anat_src, anatdir)
        system(cmdstring)
        print(cmdstring)

        # put subjectname textfile into main folder !!!!!!!!!!!!!!!!!!
        maketxt(subname, SESSION_DIR)

    except FileExistsError:
        print("directory already exists, proceeding with unpacking")
 #     continue

    full = os.listdir(dicomdir) # checks for empty directory
    if full==0:
        # copy these dicoms to the specified path
        cmdstring = "rsync %s %s" % (srcdir+'*', dicomdir)
        system(cmdstring)
        print(cmdstring)
        
    exists = os.path.isfile(SESSION_DIR+'/scan.info') # checks for scan.info file
    if exists==0:
        # unpack these dicoms
        cmdstring = 'unpacksdcmdir -src %s -targ %s -scanonly %s/scan.info' % (srcdir, SESSION_DIR, SESSION_DIR)
        system(cmdstring)
        print(cmdstring)
        #make lines low into one function
        
    exists = os.path.isfile(SESSION_DIR+'/scaninfo.csv') # checks for parsed scaninfo.csv file
    if exists==0:
        with open(SESSION_DIR+'/scan.info', 'r') as in_file:
            print("Parsing scan.info to scaninfo.csv ...")
            for line in in_file:
                # make list out of elements in each line
                editline = line.split()
                with open(SESSION_DIR+"/scaninfo.csv", 'a') as result:
                    wr = csv.writer(result, dialect='excel')
                    wr.writerow(editline)
            print("COMPLETE")
            
    # import scan.info from SESSION_DIR as pandas dataframe
    scan_info = pd.read_csv(SESSION_DIR+'/scaninfo.csv', header=None)
    
    # for each of the confirmed runs, make a bold_directory for task
    for element in task:
        # initialize indexing list to later add to cmdstrng
        run_list = []
        scan_name_index = []
        scan_num_index = []
        #runsdir = SESSION_DIR+"bold_"+element+"/"
        runsdir = "bold_"+element
        #try:
        #    os.makedirs(runsdir)
        #except FileExistsError:
        #    print("directory already exists, adding content")
        for row in range(0, len(scan_info.index)):
            # index values of every row with inclusion criteria (obain task name and run #)
            if element in scan_info.iloc[row,1]:
                # ask for confirmation of task contents (these are all the runs)
                instring = 'Would you like to include run '+str(scan_info.iloc[row,1])+' for task '+str(element)+' ?'
                userinput = verifyrun(instring)
                if userinput == 1:
                    scan_name_index.append(scan_info.iloc[row,1])
                    scan_num_index.append(scan_info.iloc[row,0])
                
        for name in range(0, len(scan_name_index)):
            temp = ' -run '+ str(scan_num_index[name]) + ' '+ runsdir+ ' ' + unpackedtype + ' ' + scan_name_index[name]+'.'+unpackedtype
            run_list.append(temp) 

        #combine list of strings into one long string about run info
        runstring = "".join(run_list)

        #add line to make sure it never re-runs if already there!!!!!!!!
        cmdstring = 'unpacksdcmdir -src %s -targ %s %s' % (srcdir, SESSION_DIR ,runstring) # directory sturture may be messed up!
        print(cmdstring)
        system(cmdstring)
        
print("------------------------- UNPACKING COMPLETE --------------------------------")



directory already exists, proceeding with unpacking
Would you like to include run fMAP_FPW_TR3_3mm_ISO_TP182 for task FPW ? [y/n]: y
unpacksdcmdir -src /cluster/archive/317/siemens/TrioTim-35162-20170110-125934-001472/ -targ /autofs/cluster/animal/users/rezzo/PCA/PCA_ITAY_F_34_20170110/  -run 7 bold_FPW nii fMAP_FPW_TR3_3mm_ISO_TP182.nii
------------------------- UNPACKING COMPLETE --------------------------------


In [12]:
scan_info

,0,1,2,3,4,5,6,7
0,1,localizer_12ch,ok,192,192,3,1,MR.1.3.12.2.1107.5.2.32.35162.2017011013083573...
1,2,AAScout,ok,128,128,128,2,MR.1.3.12.2.1107.5.2.32.35162.2017011013090637...
2,3,MEMPRAGE-vb17,ok,256,256,176,4,MR.1.3.12.2.1107.5.2.32.35162.2017011013162196...
3,4,MEMPRAGE-vb17,ok,256,256,176,1,MR.1.3.12.2.1107.5.2.32.35162.2017011013162250...
4,5,ep2d_t1w_128_2mm,ok,112,128,55,1,MR.1.3.12.2.1107.5.2.32.35162.2017011013180650...
5,6,ge_func_128_2mm_baseline_rest,ok,112,128,55,76,MR.1.3.12.2.1107.5.2.32.35162.2017011013190184...
6,7,fMAP_FPW_TR3_3mm_ISO_TP182,ok,96,96,36,182,MR.1.3.12.2.1107.5.2.32.35162.2017011013280711...
7,8,fMAP_MT_3mm_TR2_ISO_TP153,ok,96,96,32,153,MR.1.3.12.2.1107.5.2.32.35162.2017011013391980...
8,9,gre_field_mapping_DTI,ok,128,128,64,2,MR.1.3.12.2.1107.5.2.32.35162.2017011013490535...
9,10,gre_field_mapping_DTI,ok,128,128,64,1,MR.1.3.12.2.1107.5.2.32.35162.2017011013490672...


In [81]:
# find a way to create paradigm files automatically; for now, manually add

for elements in range(0, len(session_values["SUBJECT"])): # define range as number of sessions
    srcdir = session_values["PATH"][elements]+"/"
    SESSION_DIR = projectdir+"/"+session_values["SUBJECT"][elements]+'/'
    dicomdir = SESSION_DIR+"dicoms/"
    taskname = session_values["SUBJECT"][elements]
    runsdir = SESSION_DIR+"bold_"+projectname+"/"
    anatdir = SESSION_DIR+"anat/"

    if 'MT' in taskname:
        parsrc = '/autofs/cluster/animal/users/rezzo/backup_parfiles/MT_run1_paradigmFile'
        cmdstring = 'cp %s %s' % (parsrc, runsdir)
        print(cmdstring)
        system(cmdstring)
    else:
        print("task name needed")


task name needed
task name needed


In [83]:
# in terminal::
# FOR MT

# change sessidfile to have the subject of interest

# this one is linked to /space/weck/2/users/MR/ITAY/
setenv SUBJECTS_DIR /autofs/cluster/animal/users/rezzo/recons/ITAY_F_34_20170110/ 
setenv SUBJECT edit.FS6_004_019

# this one has a recursive symlink
setenv SUBJECTS_DIR /autofs/cluster/animal/users/rezzo/recons/ITAY_F_34_20171220/
setenv SUBJECT edit.FS6_004_006

mkanalysis-sess -fsd bold -native -analysis firstlevel.lh -surface $SUBJECT lh -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 25 -polyfit 2 -nskip 4 -TR 2 -nconditions 2 -force
mkanalysis-sess -fsd bold -native -analysis firstlevel.rh -surface $SUBJECT rh -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 25 -polyfit 2 -nskip 4 -TR 2 -nconditions 2 -force
mkanalysis-sess -fsd bold -native -analysis firstlevel.whole -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 25 -polyfit 2 -nskip 4 -TR 2 -nconditions 2 -force

mkcontrast-sess -analysis firstlevel.rh -contrast moving-vs-still -a 1 -c 2
mkcontrast-sess -analysis firstlevel.lh -contrast moving-vs-still -a 1 -c 2
mkcontrast-sess -analysis firstlevel.whole -contrast moving-vs-still -a 1 -c 2

# !!!!!!!!!!!!!! REPLACE SUBJECT NAME
preproc-sess -s PCA_ITAY_F_34_20170110 -fsd bold -surface $SUBJECT lhrh -per-run -fwhm 5
preproc-sess -s PCA_ITAY_F_34_20171220 -fsd bold -surface $SUBJECT lhrh -per-run -fwhm 5

# !!!!!!!!!!!!!!!!1 REPLACE SESSION ID FILE
## make a file with a list of all sessions in project folder
selxavg3-sess -sf sessidfile -analysis firstlevel.lh -overwrite
selxavg3-sess -sf sessidfile -analysis firstlevel.rh -overwrite######################################## left off here
selxavg3-sess -sf sessidfile -analysis firstlevel.whole -overwrite

# visualize surface by hemisphere
tksurfer-sess -s PCA_ITAY_F_34_20170110 -analysis firstlevel.lh -c moving-vs-still # for surface
tksurfer-scd ..

# FOR VOLUME JAN
#setenv SUBJECTS_DIR /autofs/cluster/animal/scan_data/dl-xnat/MASTER/ITAY_F_34/PCA_ITAY_F_34_20171220/recons/FS6_004_006/2017-12-21T17-10-11-115/
tkmedit-sess -s PCA_ITAY_F_34_20170110 -analysis firstlevel.whole -c moving-vs-still
# FOR VOLUME DEC
#setenv SUBJECTS_DIR /autofs/cluster/animal/scan_data/dl-xnat/MASTER/ITAY_F_34/PCA_ITAY_F_34_20171220/recons/FS6_004_006/2017-12-21T17-10-11-115/
tkmedit-sess -s PCA_ITAY_F_34_20171220 -analysis firstlevel.whole -c moving-vs-still

'PCA_ITAY_F_34_20171220'

In [ ]:
# FOR FPW

# change sessidfile to have the subject of interest


# this one has a recursive symlink
setenv SUBJECTS_DIR /autofs/cluster/animal/users/rezzo/recons/ITAY_F_34_20171220/
setenv SUBJECT edit.FS6_004_006

mkanalysis-sess -fsd bold -native -analysis firstlevel.lh -surface $SUBJECT lh -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 8 -polyfit 2 -nskip 4 -TR 2 -nconditions 4 -force
mkanalysis-sess -fsd bold -native -analysis firstlevel.rh -surface $SUBJECT rh -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 8 -polyfit 2 -nskip 4 -TR 2 -nconditions 4 -force
mkanalysis-sess -fsd bold -native -analysis firstlevel.whole -fwhm 5 -paradigm paradigm.par -event-related -spmhrf 0 -refeventdur 8 -polyfit 2 -nskip 4 -TR 2 -nconditions 4 -force

mkcontrast-sess -analysis firstlevel.rh -contrast faces-vs-placeswords -a 1 -c 4 -c 2
mkcontrast-sess -analysis firstlevel.lh -contrast faces-vs-placeswords -a 1 -c 4 -c 2
mkcontrast-sess -analysis firstlevel.whole -contrast faces-vs-placeswords -a 1 -c 4 -c 2

# mkcontrast-sess -analysis firstlevel.rh -contrast places-vs-faceswords -a 4 -c 1 -c 2
# mkcontrast-sess -analysis firstlevel.lh -contrast places-vs-faceswords -a 4 -c 1 -c 2
# mkcontrast-sess -analysis firstlevel.whole -contrast places-vs-faceswords -a 4 -c 1 -c 2

preproc-sess -s PCA_ITAY_F_34_20171220 -fsd bold -surface $SUBJECT lhrh -per-run -fwhm 5 -force

# ADD REGISTATION!!!!


# !!!!!!!!!!!!!!!!1 REPLACE SESSION ID FILE
## make a file with a list of all sessions in project folder
selxavg3-sess -sf sessidfile -analysis firstlevel.lh -overwrite
selxavg3-sess -sf sessidfile -analysis firstlevel.rh -overwrite
selxavg3-sess -sf sessidfile -analysis firstlevel.whole -overwrite

# visualize surface by hemisphere
tksurfer-sess -s PCA_ITAY_F_34_20171220 -analysis firstlevel.lh -c faces-vs-placeswords # for surface
tksurfer-sess -s PCA_ITAY_F_34_20171220 -analysis firstlevel.rh -c faces-vs-placeswords

# FOR VOLUME
#setenv SUBJECTS_DIR /autofs/cluster/animal/scan_data/dl-xnat/MASTER/ITAY_F_34/PCA_ITAY_F_34_20171220/recons/FS6_004_006/2017-12-21T17-10-11-115/
tkmedit-sess -s PCA_ITAY_F_34_20171220 -analysis firstlevel.whole -c faces-vs-placeswords



In [ ]:
# 0	0	10	1	fixation
# 10	1	8	1	faces
# 18	2	8	1	words
# 26	3	8	1	scrambled
# 34	1	8	1	faces
# 42	2	8	1	words
# 50	4	8	1	places
# 58	4	8	1	places
# 66	0	10	1	fixation
# 76	3	8	1	scrambled
# 84	0	10	1	fixation
# 94	2	8	1	words
# 102	4	8	1	places
# 110	3	8	1	scrambled
# 118	1	8	1	faces
# 126	4	8	1	places
# 134	2	8	1	words
# 142	0	10	1	fixation
# 152	1	8	1	faces
# 160	3	8	1	scrambled
# 168	2	8	1	words
# 176	0	10	1	fixation
# 186	3	8	1	scrambled
# 194	1	8	1	faces
# 202	1	8	1	faces
# 210	0	10	1	fixation
# 220	4	8	1	places
# 228	2	8	1	words
# 236	3	8	1	scrambled
# 244	4	8	1	places
# 252	0	10	1	fixation
# 262	4	8	1	places
# 270	2	8	1	words
# 278	1	8	1	faces
# 286	0	10	1	fixation
# 296	4	8	1	places
# 304	3	8	1	scrambled
# 312	3	8	1	scrambled
# 320	2	8	1	words
# 328	1	8	1	faces

In [ ]:
# 0	0	10	1	fixation
# 10	1	8	1	words
# 18	2	8	1	scrambled
# 26	3	8	1	faces
# 34	4	8	1	places
# 42	1	8	1	words
# 50	0	10	1	fixation
# 60	4	8	1	places
# 68	3	8	1	faces
# 76	2	8	1	scrambled
# 84	1	8	1	words
# 92	0	10	1	fixation
# 102	4	8	1	places
# 110	0	10	1	fixation
# 120	3	8	1	faces
# 128	2	8	1	scrambled
# 136	3	8	1	faces
# 144	1	8	1	words
# 152	2	8	1	scrambled
# 160	4	8	1	places
# 168	0	10	1	fixation
# 178	4	8	1	places
# 186	1	8	1	words
# 194	3	8	1	faces
# 202	3	8	1	faces
# 210	2	8	1	scrambled
# 218	1	8	1	words
# 226	0	10	1	fixation
# 236	2	8	1	scrambled
# 244	4	8	1	places
# 252	0	10	1	fixation
# 262	3	8	1	faces
# 270	1	8	1	words
# 278	4	8	1	places
# 286	0	10	1	fixation
# 296	2	8	1	scrambled
# 304	3	8	1	faces
# 312	2	8	1	scrambled
# 320	1	8	1	words
# 328	4	8	1	places


In [ ]:
# cd
